# Introduction to Data Science, CS 5963 / Math 3900

## Lab 19: Parallel Computing, MapReduce, and Spark. 

In this lab, we'll discuss parallel computing with a focus on MapReduce and Spark. 

**Further reading:**

[J. Lin and C. Dyer, Data-Intensive Text Processing with MapReduce (2010)](https://lintool.github.io/MapReduceAlgorithms/MapReduce-book-final.pdf)


## What is parallel computing? 

*Parallel computing* is when a computational task is broken into smaller subtasks, which are  processed simultaneously and indpendently, typically by multiple processors or computers. 

Of course, the benefits of parallel computing are that either larger tasks can be completed or tasks can be completed faster. 

**Example:** In homework 4, to use cross validation to search for optimal parameters, you could have put each parameter tested on a different computer. See the n_jobs parameter in the scikit-learn command [*GridSearchCV*](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

On the other hand, parallel computing is not easy. There are a large number of low-level programming aspects that must be handled. For example, one must consider  
- Partitioning input data
- Shared memory (Open Multiprocessing (OpenMP)) or distributed memory (Message Passing Interface (MPI)) architectures
- Scheduling execution
- Handling failures
- Interprocessor communication

There are a lot of difficult Computer Science questions here! For example, [Amdahl's law](https://en.wikipedia.org/wiki/Amdahl%27s_law) gives the theoretical speedup of a process depending on how much of the task can be parallelized. 

Python has some support for low-level parallelism; see the 
[python documentation](https://docs.python.org/3.6/library/concurrency.html), 
[multiprocessing](https://docs.python.org/3.6/library/multiprocessing.html),
[ipyparallel](https://ipyparallel.readthedocs.io/en/latest/index.html), 
[joblib](https://pypi.python.org/pypi/joblib), 
*etc*...

Today, I'll just discuss some parallelization tools built for data analysis, especially,  MapReduce. 

## MapReduce

1. Programming model for  distributed computations
+ Addressing large data sets (think ~ 1 terabyte of data)
+ Parallel and distributed algorithm
+ Cluster framework
+ Functional

History:

1. Developed by Google, but built on previously-developed ideas
+  Apache Hadoop is an open source implemenation 
+ implemented in Java
+ There are several Python interfaces to Hadoop, including MrJob, etc.... Unfortunately, these don't yet work nicely with Jupyter notebook. 

For tasks that have a particular structure, MapReduce is fairly easy to use.

- See Lecture 14 of [Harvard CS109 notes](https://drive.google.com/drive/folders/0BxYkKyLxfsNVd0xicUVDS1dIS0k)
- See the accompanying video [here](http://cs109.github.io/2015/pages/videos.html).

**Famous example: word count**

[Google Ngram viewer](https://books.google.com/ngrams) looks at word frequencies of all google books

**Exercise:** How would you use MapReduce to find anagrams?

## Spark

Spark can be thought of as  MapReduce 2.0

- In memory as opposed to disk
- Data can be cached in memory or disk for future use
- 100x faster than Hadoop MapReduce in memory or 10x faster on disk
- resilient distributed dataset (RDD)
- Python, Java, and Scala interfaces
- [apache-spark](http://spark.apache.org/) can be used in python through [findspark](https://github.com/minrk/findspark)
- Easier than Hadoop while being functional, runs a general DAG

For more, see Lecture 15 of [Harvard CS109 notes](https://drive.google.com/drive/folders/0BxYkKyLxfsNVd0xicUVDS1dIS0k) 
and accompanying [notebook](https://github.com/cs109/2015/blob/master/Lectures/15b-Spark.ipynb)

Spark must be installed seperately than Anaconda. For Mac users, try

```
$ brew install apache-spark
$ pip install findspark
```

In [1]:
import numpy as np

import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext()

**Example:** Compute the sum $\sum_{i=1}^{10} i^2$ 

We could do this using numpy as follows

In [2]:
np.sum(np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])**2)

385

Using pyspark, here's how to evaluate the sum. The second parameter below specifies the number of partitions to cut the dataset into. Spark will run one task for each partition. Typically, you can choose 2-4 partitions for each CPU in your cluster. Spark will automatically pick this number if you don't specify  it. 

In [5]:
sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10],4).map(lambda x: x**2).sum()

385

Finally, we can count the words in a list. 

In [8]:
wordsList = ['cat', 'elephant', 'rat', 'rat', 'cat']
wordsRDD = sc.parallelize(wordsList, 4)      
wordCountsCollected = (wordsRDD
                       .map(lambda w: (w, 1))
                       .reduceByKey(lambda x,y: x+y)
                       .collect())
print(wordCountsCollected)

[('cat', 2), ('elephant', 1), ('rat', 2)]


### Amazon Web Service (AWS)

[Amazon Web Service (AWS)](https://aws.amazon.com/) offeres cloud-computing services that make up an on-demand computing platform.

